In [2]:
pip install pandas plotly

In [1]:
import pandas as pd

df = pd.read_csv('data/0_preprocessed.csv')
df

,timestamp,duration,song,artist,album,song_artist
0,2019-07-05 03:00:26,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,2019-07-05 03:01:44,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,2019-07-05 03:05:23,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,2019-07-05 03:08:41,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,2019-07-05 03:12:01,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,2024-09-04 13:36:23,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,2024-09-04 14:35:32,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,2024-09-05 13:50:59,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,2024-09-05 16:08:47,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [2]:
from datetime import datetime, timedelta

# Define the start and end dates
start_date = datetime(2019, 7, 5)
end_date = datetime(2024, 9, 12)

# Create an empty list to store the timestamps
timestamps = []

# Generate timestamps with a daily increment
current_date = start_date
while current_date <= end_date:
  timestamps.append(current_date)
  current_date += timedelta(weeks=1)

# Print the list of timestamps
print(len(timestamps), timestamps[0], timestamps[-1])

271 2019-07-05 00:00:00 2024-09-06 00:00:00


In [10]:
gantt_data = []

# Iterate over the timestamps
for i in range(len(timestamps) - 1):
  # Get the current timestamp and the next timestamp
  current_timestamp = timestamps[i]
  next_timestamp = timestamps[i + 1]

  # Filter the DataFrame based on the timestamps
  filtered_df = df[(df['timestamp'] >= str(current_timestamp)) & (df['timestamp'] < str(next_timestamp))]

  # Calculate the total duration for each artist
  artist_duration = filtered_df.groupby('artist')['duration'].sum()

  # Find the artist with the most played time
  most_played_artist = artist_duration.idxmax() if not artist_duration.empty else None

  # Calculate the total duration for each song
  song_duration = filtered_df.groupby('song')['duration'].sum()

  # Find the song with the most played time
  most_played_song = song_duration.idxmax() if not song_duration.empty else None

  # Append the data to the list
  gantt_data.append({
      'timestamp_start': current_timestamp,
      'timestamp_end': next_timestamp,
      'most_played_artist': most_played_artist,
      'most_played_song': most_played_song,
      'artist_duration': artist_duration.max() if not artist_duration.empty else 0,
      'song_duration': song_duration.max() if not song_duration.empty else 0
  })

# Create a new DataFrame from the data
gantt_df = pd.DataFrame(gantt_data)

# Print the new DataFrame
gantt_df

,timestamp_start,timestamp_end,most_played_artist,most_played_song,artist_duration,song_duration
0,2019-07-05,2019-07-12,Alan Walker,All Falls Down,69.29,50.43
1,2019-07-12,2019-07-19,Taylor Swift,Delicate,189.50,95.59
2,2019-07-19,2019-07-26,Taylor Swift,Everything Has Changed,653.72,354.68
3,2019-07-26,2019-08-02,Taylor Swift,Everything Has Changed,201.40,127.06
4,2019-08-02,2019-08-09,Taylor Swift,Everything Has Changed,265.28,95.69
...,...,...,...,...,...,...
265,2024-08-02,2024-08-09,Sơn Tùng M-TP,Nuvole Bianche,113.10,29.99
266,2024-08-09,2024-08-16,Sơn Tùng M-TP,Nơi Này Có Anh,246.20,67.74
267,2024-08-16,2024-08-23,Sơn Tùng M-TP,Nơi Này Có Anh,548.60,209.35
268,2024-08-23,2024-08-30,Sơn Tùng M-TP,Chắc Ai Đó Sẽ Về,1460.46,403.46


In [11]:
len(gantt_df['most_played_artist'].unique()), len(gantt_df['most_played_song'].unique())

(24, 74)

In [12]:
# prompt: in gantt_df, replace every value in 'most_played_artist' and 'most_played_artist' with '13 others' and '65 others' if the value is not in the top 9

# Get the top 9 most played artists
top_9_artists = gantt_df['most_played_artist'].value_counts().head(9).index

# Get the top 9 most played songs
top_9_songs = gantt_df['most_played_song'].value_counts().head(9).index

# Replace values not in the top 9 with "13 others" for artists
gantt_df['most_played_artist'] = gantt_df['most_played_artist'].apply(lambda x: '13 More' if x not in top_9_artists else x)

# Replace values not in the top 9 with "65 others" for songs
gantt_df['most_played_song'] = gantt_df['most_played_song'].apply(lambda x: '65 More' if x not in top_9_songs else x)

gantt_df


,timestamp_start,timestamp_end,most_played_artist,most_played_song,artist_duration,song_duration
0,2019-07-05,2019-07-12,13 More,65 More,69.29,50.43
1,2019-07-12,2019-07-19,Taylor Swift,65 More,189.50,95.59
2,2019-07-19,2019-07-26,Taylor Swift,65 More,653.72,354.68
3,2019-07-26,2019-08-02,Taylor Swift,65 More,201.40,127.06
4,2019-08-02,2019-08-09,Taylor Swift,65 More,265.28,95.69
...,...,...,...,...,...,...
265,2024-08-02,2024-08-09,Sơn Tùng M-TP,Nuvole Bianche,113.10,29.99
266,2024-08-09,2024-08-16,Sơn Tùng M-TP,Nơi Này Có Anh,246.20,67.74
267,2024-08-16,2024-08-23,Sơn Tùng M-TP,Nơi Này Có Anh,548.60,209.35
268,2024-08-23,2024-08-30,Sơn Tùng M-TP,65 More,1460.46,403.46


In [64]:
gantt_top_artists = gantt_df['most_played_artist'].value_counts()
gantt_top_artists

,count
most_played_artist,
Taylor Swift,143
Ludovico Einaudi,72
13 More,20
Sơn Tùng M-TP,15
Westlife,7
Joe Hisaishi,3
Lana Del Rey,3
Andiez,3
Justin Bieber,2


In [65]:
gantt_top_songs = gantt_df['most_played_song'].value_counts()
gantt_top_songs

,count
most_played_song,
65 More,106
Nuvole Bianche,66
Enchanted,41
Sparks Fly,14
Nơi Này Có Anh,13
Nuvole bianche,8
The Portrait,7
a thousand years,5
Moonlight Sonata,5


In [36]:
import plotly.figure_factory as ff
import plotly.express as px

In [58]:
colors = px.colors.qualitative.Bold
colors_songs = {song : color for song, color in zip(gantt_df['most_played_song'].value_counts().index, colors)}
colors_artists = {artist : color for artist, color in zip(gantt_df['most_played_artist'].value_counts().index, colors)}

In [66]:
gantt_songs, gantt_artists = [], []
for i, row in gantt_df.iterrows():
    song = row['most_played_song']
    artist = row['most_played_artist']
    start = row['timestamp_start']
    finish = row['timestamp_end']
    n_weeks_song = gantt_top_songs[song]
    n_weeks_artist = gantt_top_artists[artist]
    task_song = song + f' ({n_weeks_song}w)'
    task_artist = artist + f' ({n_weeks_artist}w)'

    gantt_songs.append(
        dict(
            Task=task_song,
            Start=start,
            Finish=finish,
            Resource=song
        )
    )

    gantt_artists.append(
        dict(
            Task=task_artist,
            Start=start,
            Finish=finish,
            Resource=artist
        )
    )

In [77]:
fig_songs = ff.create_gantt(gantt_songs, title='Most-played Song of the Week',colors=colors_songs, index_col='Resource', group_tasks=True, height=500, bar_width=0.5)
fig_songs.show()

In [78]:
fig_artists = ff.create_gantt(gantt_artists, title='Most-played Artist of the Week',colors=colors_artists, index_col='Resource', group_tasks=True, height=500, bar_width=0.5)
fig_artists.show()

In [79]:
gantt_df.to_csv('3_top_song_artist_otw.csv')